In [1]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from collections import Counter

In [2]:


# Load the dataset
df = pd.read_csv("adni_data_cleaned_with_just_dx_and_snp.csv")

# Separate features (SNPs) and target (DX)
X = df.drop(columns=["DX"])
y = df["DX"]

# Define the desired class counts
desired_counts = {0: 363, 1: 363, 2: 363}

# Apply SMOTE with the desired sampling strategy
smote = SMOTE(sampling_strategy=desired_counts, random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Round the synthetic samples to ensure valid SNP values (0, 1, 2)
X_res = np.round(X_res).astype(int)

# Reconstruct the balanced DataFrame
balanced_df = pd.DataFrame(X_res, columns=X.columns)
balanced_df["DX"] = y_res

# Shuffle the final dataset
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to CSV
balanced_df.to_csv("adni_data_balanced.csv", index=False)

# Print the final class distribution
print("Final class distribution:")
print(Counter(balanced_df["DX"]))

Final class distribution:
Counter({2: 363, 1: 363, 0: 363})


In [3]:
#value counts
print("Value counts of DX in the balanced dataset:")
print(balanced_df["DX"].value_counts())

Value counts of DX in the balanced dataset:
DX
2    363
1    363
0    363
Name: count, dtype: int64


In [ ]:
# split the dataset into features and target
X = balanced_df.drop(columns=["DX"])
y = balanced_df["DX"]
# Print the shape of the balanced dataset
print("Shape of the balanced dataset:")
print(X.shape, y.shape)

Shape of the balanced dataset:
(1089, 29) (1089,)


In [8]:
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
# import shap

# Load original dataset
df = pd.read_csv("adni_data_cleaned_with_just_dx_and_snp.csv")

# Check original class distribution
print("Original class distribution:")
print(df["DX"].value_counts())

# Separate features and target
X = df.drop(columns=["DX"])
y = df["DX"]

# Apply SMOTE to balance classes
smote = SMOTE(sampling_strategy={0: 363, 1: 363, 2: 363}, random_state=42)
X_res, y_res = smote.fit_resample(X, y)

# Round synthetic samples to valid SNP values (0, 1, 2)
X_res = np.round(X_res).astype(int)

# Create balanced DataFrame
balanced_df = pd.DataFrame(X_res, columns=X.columns)
balanced_df["DX"] = y_res

# Final class distribution check
print("\nFinal class distribution after SMOTE:")
print(balanced_df["DX"].value_counts())

# Split into features and target as specified
X = balanced_df.drop(columns=["DX"])
y = balanced_df["DX"]

# Stratified train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Define models with hyperparameter grids
models = {
    "Logistic": {
        "model": LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000),
        "params": {"C": np.logspace(-4, 4, 9)}
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=42),
        "params": {
            "n_estimators": [100, 200],
            "max_depth": [None, 10, 20],
            "min_samples_split": [2, 5]
        }
    },
    "XGBoost": {
        "model": XGBClassifier(use_label_encoder=False, eval_metric='mlogloss'),
        "params": {
            "n_estimators": [100, 200],
            "learning_rate": [0.01, 0.1],
            "max_depth": [3, 6]
        }
    },
    "SVM": {
        "model": SVC(probability=True, random_state=42),
        "params": {
            "C": [0.1, 1, 10],
            "kernel": ['rbf', 'linear']
        }
    }
}

# Train and evaluate models
results = {}

for name, config in models.items():
    print(f"\nTraining {name}...")
    
    # Grid search with cross-validation
    grid = GridSearchCV(
        config["model"], 
        config["params"],
        cv=StratifiedKFold(5),
        scoring='accuracy',
        n_jobs=-1
    )
    
    grid.fit(X_train, y_train)
    
    # Best model predictions
    best_model = grid.best_estimator_
    y_pred = best_model.predict(X_test)
    y_proba = best_model.predict_proba(X_test)
    
    # Get classification report as dict
    report_dict = classification_report(y_test, y_pred, output_dict=True)
    # Handle both possible keys for weighted average
    weighted_key = 'weighted_avg' if 'weighted_avg' in report_dict else 'weighted avg'
    # Store results
    results[name] = {
        "best_params": grid.best_params_,
        "accuracy": accuracy_score(y_test, y_pred),
        "f1_macro": report_dict[weighted_key]['f1-score'],
        "roc_auc": roc_auc_score(y_test, y_proba, multi_class='ovr')
    }
    
    # Print metrics
    print(f"Best params: {grid.best_params_}")
    print(f"Accuracy: {results[name]['accuracy']:.4f}")
    print(f"F1 Score (macro): {results[name]['f1_macro']:.4f}")
    print(f"ROC AUC: {results[name]['roc_auc']:.4f}")
    print(classification_report(y_test, y_pred))

# Feature importance analysis for tree-based models
tree_models = ["RandomForest", "XGBoost"]

for model_name in tree_models:
    if model_name in results:
        print(f"\nFeature Importance for {model_name}:")
        model = models[model_name]["model"].set_params(**results[model_name]["best_params"])
        model.fit(X_train, y_train)
        
        # Get feature importances
        importances = model.feature_importances_
        feat_imp = pd.Series(importances, index=X.columns).sort_values(ascending=False)
        print(feat_imp.head(10))
        
        # SHAP values for XGBoost
        # if model_name == "XGBoost":
        #     explainer = shap.Explainer(model)
        #     shap_values = explainer(X_train)
        #     shap.summary_plot(shap_values, X_train, plot_type="bar")

# Compare all models
print("\nModel Comparison:")
comparison_df = pd.DataFrame(results).T
print(comparison_df[["accuracy", "f1_macro", "roc_auc"]])

# Save balanced dataset
balanced_df.to_csv("balanced_adni_data.csv", index=False)

Original class distribution:
DX
1    363
0    214
2    180
Name: count, dtype: int64

Final class distribution after SMOTE:
DX
0    363
2    363
1    363
Name: count, dtype: int64

Training Logistic...


/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/

Best params: {'C': 0.001}
Accuracy: 0.4174
F1 Score (macro): 0.4163
ROC AUC: 0.6142
              precision    recall  f1-score   support

           0       0.41      0.39      0.40        72
           1       0.43      0.48      0.45        73
           2       0.42      0.38      0.40        73

    accuracy                           0.42       218
   macro avg       0.42      0.42      0.42       218
weighted avg       0.42      0.42      0.42       218


Training RandomForest...
Best params: {'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 200}
Accuracy: 0.6514
F1 Score (macro): 0.6509
ROC AUC: 0.8381
              precision    recall  f1-score   support

           0       0.61      0.69      0.65        72
           1       0.55      0.51      0.53        73
           2       0.80      0.75      0.77        73

    accuracy                           0.65       218
   macro avg       0.65      0.65      0.65       218
weighted avg       0.65      0.65      0.65      

/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:36:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:36:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:36:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:36:20] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
/home/mohamed-me

Best params: {'learning_rate': 0.1, 'max_depth': 6, 'n_estimators': 200}
Accuracy: 0.6514
F1 Score (macro): 0.6535
ROC AUC: 0.8026
              precision    recall  f1-score   support

           0       0.65      0.65      0.65        72
           1       0.56      0.62      0.59        73
           2       0.76      0.68      0.72        73

    accuracy                           0.65       218
   macro avg       0.66      0.65      0.65       218
weighted avg       0.66      0.65      0.65       218


Training SVM...
Best params: {'C': 10, 'kernel': 'rbf'}
Accuracy: 0.6239
F1 Score (macro): 0.6201
ROC AUC: 0.8239
              precision    recall  f1-score   support

           0       0.60      0.65      0.63        72
           1       0.61      0.49      0.55        73
           2       0.65      0.73      0.69        73

    accuracy                           0.62       218
   macro avg       0.62      0.62      0.62       218
weighted avg       0.62      0.62      0.62    

/home/mohamed-megdoud/anaconda3/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [10:36:33] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


rs10012882_T    0.053934
rs29745_A       0.044828
rs440277_G      0.041725
rs6882046_A     0.040157
rs8106922_A     0.040022
rs1448284_T     0.038345
rs1923775_T     0.038343
rs2718058_A     0.038026
rs9832461_A     0.037723
rs9381563_T     0.037115
dtype: float32

Model Comparison:
              accuracy  f1_macro   roc_auc
Logistic      0.417431  0.416346  0.614244
RandomForest  0.651376  0.650864  0.838064
XGBoost       0.651376  0.653482    0.8026
SVM           0.623853  0.620114  0.823883


In [ ]:
import pandas as pd
import numpy as np
from imblearn.pipeline import Pipeline as imbpipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer, Categorical
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv("adni_data_cleaned_with_just_dx_and_snp.csv")
X = df.drop(columns=["DX"])
y = df["DX"]

# Split first to prevent leakage
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Define preprocessing
preprocessor = ColumnTransformer([
    ('poly', PolynomialFeatures(degree=2, include_bias=False), X.columns),
    ('pca', PCA(n_components=0.95), X.columns)
])

# Create pipeline template
def make_pipeline(model, smote=True):
    steps = []
    if smote:
        steps.append(('smote', SMOTE(sampling_strategy={0: 363, 1: 363, 2: 363}, random_state=42)))
    steps.extend([
        ('scaler', StandardScaler()),
        ('feature_selector', SelectKBest(f_classif, k=20)),
        ('model', model)
    ])
    return imbpipeline(steps)

# Define models with expanded hyperparameter spaces
models = {
    "Logistic": {
        'pipeline': make_pipeline(LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)),
        'params': {
            'model__C': Real(1e-5, 1e5, prior='log-uniform'),
            'feature_selector__k': Integer(10, 30)
        }
    },
    
    "RF": {
        'pipeline': make_pipeline(RandomForestClassifier(random_state=42)),
        'params': {
            'model__n_estimators': Integer(100, 500),
            'model__max_depth': Integer(3, 20),
            'model__min_samples_split': Integer(2, 11),
            'model__class_weight': Categorical(['balanced', 'balanced_subsample'])
        }
    },
    
    "XGBoost": {
        'pipeline': make_pipeline(XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')),
        'params': {
            'model__n_estimators': Integer(50, 300),
            'model__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'model__max_depth': Integer(3, 12),
            'model__subsample': Real(0.6, 1.0),
            'model__colsample_bytree': Real(0.6, 1.0)
        }
    },
    
    "LGBM": {
        'pipeline': make_pipeline(LGBMClassifier(random_state=42)),
        'params': {
            'model__n_estimators': Integer(50, 300),
            'model__learning_rate': Real(0.01, 0.3, prior='log-uniform'),
            'model__num_leaves': Integer(10, 200),
            'model__feature_fraction': Real(0.6, 1.0),
            'model__bagging_fraction': Real(0.6, 1.0)
        }
    },
    
    "Stacking": {
        'pipeline': None,  # Will be defined after individual models are trained
        'params': {}
    }
}

# Train base models with Bayesian optimization
best_models = {}
results = {}

for name in ["Logistic", "RF", "XGBoost", "LGBM"]:
    print(f"\nTraining {name}...")
    
    # Create and fit optimizer
    opt = BayesSearchCV(
        models[name]['pipeline'],
        models[name]['params'],
        n_iter=50,
        cv=StratifiedKFold(5),
        scoring='accuracy',
        n_jobs=-1,
        random_state=42
    )
    
    opt.fit(X_train, y_train)
    
    # Store results
    best_models[name] = opt
    y_pred = opt.predict(X_test)
    y_proba = opt.predict_proba(X_test)
    
    results[name] = {
        "best_params": opt.best_params_,
        "accuracy": accuracy_score(y_test, y_pred),
        "f1_macro": classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'],
        "roc_auc": roc_auc_score(y_test, y_proba, multi_class='ovr')
    }
    
    print(f"{name} Accuracy: {results[name]['accuracy']:.4f}")
    print(classification_report(y_test, y_pred))

# Create stacking ensemble
estimators = [
    ('rf', best_models['RF'].best_estimator_),
    ('xgb', best_models['XGBoost'].best_estimator_),
    ('lgbm', best_models['LGBM'].best_estimator_)
]

stacking = Pipeline([
    ('smote', SMOTE(sampling_strategy={0: 363, 1: 363, 2: 363}, random_state=42)),
    ('scaler', StandardScaler()),
    ('stack', VotingClassifier(estimators, voting='soft'))
])

stacking.fit(X_train, y_train)
y_pred = stacking.predict(X_test)
y_proba = stacking.predict_proba(X_test)

results["Stacking"] = {
    "best_params": "Ensemble of best models",
    "accuracy": accuracy_score(y_test, y_pred),
    "f1_macro": classification_report(y_test, y_pred, output_dict=True)['weighted avg']['f1-score'],
    "roc_auc": roc_auc_score(y_test, y_proba, multi_class='ovr')
}

# Feature importance analysis
print("\nTop Features:")
for name in ["RF", "XGBoost", "LGBM"]:
    feat_imp = pd.Series(best_models[name].best_estimator_.named_steps['model'].feature_importances_, 
                        index=X.columns).sort_values(ascending=False)
    print(f"\n{name} Top 10 Features:")
    print(feat_imp.head(10))

# Compare results
comparison_df = pd.DataFrame(results).T
print("\nModel Comparison:")
print(comparison_df[["accuracy", "f1_macro", "roc_auc"]])